In [ ]:
; !pip install transformers
!pip install torchmetrics torch
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from torchmetrics  import Accuracy
import re
from transformers import BertForSequenceClassification, AutoTokenizer
import tqdm
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
dataset=load_dataset('imdb')
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
def pre_process(data):
  data=re.sub("http/s"," ",data)
  data=re.sub("[^a-zA-Z]"," ",data)
  data=data.lower()
  return data

In [ ]:
def pipeline(dataframe):
  dataframe['text']=dataframe['text'].apply(pre_process)
  sentences=['[cls]'+ s for s in dataframe['text'].tolist()]
  tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
  Tokens=[tokenizer.tokenize(s) for s in sentences]
  max_length=140
  Tokens=[t[:139]+['sep'] for t in Tokens]
  ids=[tokenizer.convert_tokens_to_ids(t) for t in Tokens]
  ids=np.array([np.pad(i,(0, max_length-len(i)),mode='constant') for i in ids])
  amsaks=np.asarray([[float(i>0) for i in s]for s in ids])

  lables=dataframe['label'].tolist()
  print(amsaks)
  return torch.tensor(ids), torch.tensor(lables), torch.tensor(amsaks)

In [ ]:
d_train=dataset['train'].to_pandas().head(20000)
d_test=dataset['test'].to_pandas()
ids,labels,amsaks=pipeline(d_train)
tensordataset=TensorDataset(ids,labels,amsaks)
loader=DataLoader(tensordataset,batch_size=32,shuffle=True)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (570 > 512). Running this sequence through the model will result in indexing errors


[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 1. 1. 1.]]


In [ ]:
model=BertForSequenceClassification.from_pretrained("bert-base-uncased")
model.classifier=torch.nn.Linear(768,5)

model.to(device)
for i, (names,params) in enumerate(model.named_parameters()):
  if "classifier" in names:
    params.requires_grad=True
  else:
    params.requires_grad=False

max_param=0
for params in model.parameters():
   if params.requires_grad:
    max_param+=params.numel()
print(max_param)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


3845


In [ ]:
optimizer=torch.optim.AdamW(model.parameters(),lr=0.0003, eps=1e-8)
cross_entropy=torch.nn.CrossEntropyLoss()
Accur_cal=Accuracy(task="multiclass", num_classes=5).to(device)
epoch=2

model.train()
loss=list()
for i in range(epoch):
  for idx_t,label_t,amasks_t in loader:
    optimizer.zero_grad()
    idx_t,label_t,amasks_t=idx_t.to(device),label_t.to(device),amasks_t.to(device)
    pred=model(idx_t, attention_mask=amasks_t)
    Accur_cal.update(torch.argmax(pred.logits,dim=1),label_t)
    loss_t=cross_entropy(pred.logits,label_t)
    loss.append(loss_t.item())
    loss_t.backward()
    optimizer.step()


print(Accur_cal.compute().item())
print(loss_t)

0.6478999853134155


In [ ]:
loss_t

tensor(0.5089, device='cuda:0', grad_fn=<NllLossBackward0>)